# GenAI Career Assistant Agent – Your Ultimate Guide to a Career in Generative AI!🚀

## Modificaciones para usar Groq en lugar de Gemini

In [ ]:
# Importaciones actualizadas
from typing import Dict, TypedDict
from langgraph.graph import StateGraph, END, START
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq  # Reemplazamos ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor

from IPython.display import display, Image, Markdown
from langchain_core.runnables.graph import MermaidDrawMethod
from dotenv import load_dotenv
from datetime import datetime

import os

In [ ]:
# Cargar variable de entorno para la API de Groq
load_dotenv()

# Configurar la clave de API de Groq
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY', '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')

# Inicializar el modelo Groq (usando LLaMA3 70b)
llm = ChatGroq(
    model="llama3-70b-8192",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0.5,
    verbose=True
)

## Definición de Estado y Utilidades

In [ ]:
# Definición de estado y funciones de utilidad
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, trim_messages

class State(TypedDict):
    query: str
    category: str
    response: str

def trim_conversation(prompt):
    """Trims conversation history to retain only the latest messages within the limit."""
    max_messages = 10  # Limit the conversation history to the latest 10 messages
    return trim_messages(
        prompt,
        max_tokens=max_messages,  # Specifies the maximum number of messages allowed
        strategy="last",  # Trimming strategy to keep the last messages
        token_counter=len,  # Counts tokens/messages using the length of the list
        start_on="human",  # Start trimming when reaching the first human message
        include_system=True,  # Include system messages in the trimmed history
        allow_partial=False,  # Ensures only whole messages are included
    )

def save_file(data, filename):
    """Saves data to a markdown file with a timestamped filename."""
    folder_name = "Agent_output"  # Folder to store output files
    os.makedirs(folder_name, exist_ok=True)  # Creates the folder if it doesn't exist
    
    # Generate a timestamped filename for uniqueness
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")  # Format: YYYYMMDDHHMMSS
    filename = f"{filename}_{timestamp}.md"
    
    # Define the full file path
    file_path = os.path.join(folder_name, filename)
    
    # Save the data to the file in the specified path
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(data)
        print(f"File '{file_path}' created successfully.")
    
    # Return the full path of the saved file
    return file_path

def show_md_file(file_path):
    """Displays the content of a markdown file as Markdown in the notebook."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Render the content in Markdown format within the notebook
    display(Markdown(content))

## Clase de Recursos de Aprendizaje

In [ ]:
class LearningResourceAgent:
    def __init__(self, prompt):
        # Inicializar el modelo de chat con Groq
        self.model = ChatGroq(model="llama3-70b-8192")
        self.prompt = prompt
        self.tools = [DuckDuckGoSearchResults()]

    def TutorialAgent(self, user_input):
        # Configurar un agente con acceso a herramientas y ejecutar una respuesta de tutorial
        agent = create_tool_calling_agent(self.model, self.tools, self.prompt)
        agent_executor = AgentExecutor(agent=agent, tools=self.tools, verbose=True)
        response = agent_executor.invoke({"input": user_input})
        
        # Guardar y mostrar la respuesta como archivo markdown
        path = save_file(str(response.get('output')).replace("```markdown", "").strip(), 'Tutorial')
        print(f"Tutorial guardado en {path}")
        return path

    def QueryBot(self, user_input):
        # Iniciar un bucle de preguntas y respuestas para interacción continua con el usuario
        print("\nIniciando la sesión de preguntas y respuestas. Escribe 'exit' para terminar la sesión.\n")
        record_QA_session = []
        record_QA_session.append(f'Consulta del usuario: {user_input} \n')
        self.prompt.append(HumanMessage(content=user_input))

        while True:
            # Recortar el historial de conversación para mantener el tamaño del prompt
            self.prompt = trim_conversation(self.prompt)
            
            # Generar una respuesta del modelo de IA y actualizar el historial de conversación
            response = self.model.invoke(self.prompt)
            record_QA_session.append(f'\nRespuesta del experto: {response.content} \n')
            
            self.prompt.append(AIMessage(content=response.content))
            
            # Mostrar la respuesta de la IA y solicitar entrada del usuario
            print('*' * 50 + 'AGENTE' + '*' * 50)
            print("\nRESPUESTA DEL AGENTE EXPERTO:", response.content)
            
            print('*' * 50 + 'USUARIO' + '*' * 50)
            user_input = input("\nSU CONSULTA: ")
            record_QA_session.append(f'\nConsulta del usuario: {user_input} \n')
            self.prompt.append(HumanMessage(content=user_input))
            
            # Salir del bucle de preguntas y respuestas si el usuario escribe 'exit'
            if user_input.lower() == "exit":
                print("Terminando la sesión de chat.")
                path = save_file(''.join(record_QA_session), 'Sesion_Preguntas_Dudas')
                print(f"Sesión de preguntas guardada en {path}")
                return path